# 目标:使用QT编写一个TCP服务器和一个TCP客户端

## 网络协议
网络协议是指通信双方就通信如何进行所必须共同遵守的约定和通信规则的集合。在网络上通信的双方只有遵守相同的协议，才能正确地交流信息，就像人们交谈时要使用同一种语言一样，如果谈话里使用不同的语言，就会造成双方都不知所云，交流就被迫中断。典型的网络协议有：TCP/IP协议、IPX/SPX协议、IEEEE802标准协议系列、X.25协议等。 

## 传输层
综合OSI和TCP/IP参考模型的优点，采用一种5层的网络体系结构。传输层即五层网络体系结构中的一层。传输层的设计目标是允许源主机和目标主机上的对等实体进行对话，即为两个主机中进程之间的通信提供服务。例如，一台主机上的浏览器进程与另一台主机上的Web服务器进程之间进行通信。
传输层的基本功能是接收来自上一层应用层的数据，在必要的时候把这些数据分割成较小的单元，然后把这些数据单元传递给网络层，并且确保这些数据单元能够正确地到达另一端。
传输层是真正的端到端的层，它负责将数据从源端传送到目标端，即源端主机上的一个程序利用传输层协议与目标端主机上的一个程序进行会话。而在其下面的各层，只涉及一台主机与它的直接邻居的通信，这是因为源主机和目标主机之间可能存在多个中间路由器。

在TCP/IP网络体系结构中，TCP（传输控制协议，Transport Control Protocol、UDP（用户数据报协议，User Data Protocol）是传输层最重要的两种协议，为上层用户提供级别的通信可靠性。

### 传输控制协议（TCP）：
TCP（传输控制协议）定义了两台计算机之间进行可靠的传输而交换的数据和确认信息的格式，以及计算机为了确保数据的正确到达而采取的措施。协议规定了TCP软件怎样识别给定计算机上的多个目的进程如何对分组重复这类差错进行恢复。协议还规定了两台计算机如何初始化一个TCP数据流传输以及如何结束这一传输。TCP最大的特点就是提供的是面向连接、可靠的字节流服务。
### 用户数据报协议（UDP）：
UDP（用户数据报协议）是一个简单的面向数据报的传输层协议。提供的是非面向连接的、不可靠的数据流传输。UDP不提供可靠性，也不提供报文到达确认、排序以及流量控制等功能。它只是把应用程序传给IP层的数据报发送出去，但是并不能保证它们能到达目的地。因此报文可能会丢失、重复以及乱序等。但由于UDP在传输数据报前不用在客户和服务器之间建立一个连接，且没有超时重发等机制，故而传输速度很快。


![](./image/network_struct.png)



## 环境准备

我们首先使用工具在本地建立一个TCP服务器和TCP客户端,并且模拟互相通讯.

链接：https://pan.baidu.com/s/1S8HjvjhEYXwVOGx8maVgkg 
提取码：wiy0 


## 模拟环境
下载成功后,启动两个同样的程序,分别设置为TCP服务器和TCP客户端,为服务器设定监听的端口为8888,然后点击"启用"复选框启动服务器;为客户端设置服务器IP为127.0.0.1,端口为8888,然后点击"启用"复选框启动客户端.
当客户端正常连接服务器后,互相发送数据,并查看对方是否收到对应的数据.

![](./image/clientandserver.png)



# 用QT实现TCP客户端

上面已经让大家尝试过服务器和客户端如何通讯了,下一步我们用QT来写一个客户端.
在我们开始写程序之前首先了解一下什么是Socket:

在组网领域的首次使用是在1970年2月12日发布的文献IETF RFC33中发现的，撰写者为Stephen Carr、Steve Crocker和Vint Cerf。根据美国计算机历史博物馆的记载，Croker写道：“命名空间的元素都可称为套接字接口。一个套接字接口构成一个连接的一端，而一个连接可完全由一对套接字接口规定。”计算机历史博物馆补充道：“这比BSD的套接字接口定义早了大约12年。”

在计算机通信领域，socket 被翻译为“套接字”，它是计算机之间进行通信的一种约定或一种方式。通过 socket 这种约定，一台计算机可以接收其他计算机的数据，也可以向其他计算机发送数据。


## 增加网络库
首先新建一个项目,并且修改项目的.pro文件:

```
greaterThan(QT_MAJOR_VERSION, 4): QT += widgets network
```



## 建立一个Socket类

```
class TcpClient : public QMainWindow
{
public:
    TcpClient(QWidget *parent = 0) : QMainWindow(parent)
    {
        tcpClient = new QTcpSocket(this);   //实例化tcpClient
    }

    ~TcpClient(){
        delete tcpClient;
    }
public:
    QHostAddress ip;
    quint16 port;

public:
    QTcpSocket *tcpClient;//套接字
    void ConServer();//连接服务器
    qint64 SentData(const char* data, qint64 len);//发送数据
    void DisconServer();//断开连接
};
```

我们从这个类的定义可以看出来我们需要一个QTcpSocket类型的变量来进行忘了操作,同时我们定义了三个函数分别进行连接服务器,发送数据和断开服务器连接.

## 断开服务器连接

```
void TcpClient::DisconServer()
{
    qDebug()<<"try to disconnect.....";
    tcpClient->disconnectFromHost();
    if (tcpClient->state() == QAbstractSocket::UnconnectedState \
            || tcpClient->waitForDisconnected(1000))  //已断开连接则进入if{}
    {
        qDebug()<<"disconnected!";
        return;
    }
    qDebug()<<"disconnect failed";
}
```

断开服务器连接和建立服务器连接是一个相反的过程.同样断开的过程也需要一段时间,所以我们也需要调用waitForDisconnected来等待服务器断开的结果.

当服务器断开后,socket的状态会自动更新为:UnconnectedState


## 向服务器发送数据

```
qint64 TcpClient::SentData(const char* data, qint64 len)
{
    qDebug()<<"sent data.....";
    qint64 slen=tcpClient->write(data,len);
    qDebug()<<"send "<<slen<<" bytes";
    return slen;
}
```

当客户端和服务器之间通过socket建立连接后,客户端就可以随时通过socket发送和接收数据.我们先看比较简单的发送数据.
从逻辑上,套接字就像是一个管道,两端分别连接服务器和客户端.操作这个管道的时候,类似于文件操作的思想.也就是说如果需要发送数据就write数据到套接字,读取数据就从套接字read数据.
当write或者read后,套接字会返回本次实际上发送和接收到了多少个数据.

需要注意的是因为TCP协议本身对于一次发送数据包大小的限制,所以并不能直接写入一个非常大文件.对于比较大的数据需要分多次发送和接收.



## 验证向服务器发送数据

完成上面的工作后,我们就可以向服务器发送数据了.首先保持我们之前建立的演示环境,特别是服务器端开启在监听状态.
然后我们建立一个上面TcpClient类的对象并进行必要的设置.

```
int main(int argc, char *argv[])
{
    char data[]={(char)0x38,(char)0x36,(char)0x37,(char)0x35};
    QApplication a(argc, argv);
    TcpClient tc;
    tc.ip= "127.0.0.1";
    tc.port=8888;//8989;
    //qDebug()<<tc.tcpClient->state();
    tc.ConServer();
    //qDebug()<<tc.tcpClient->state();
    if(tc.tcpClient->state() == QAbstractSocket::ConnectedState)
        tc.SentData(data,sizeof(data));
    
    tc.DisconServer();

    return a.exec();
}
```

在上面的程序中,我们设置了服务器IP地址为127.0.0.1,这个IP地址是一个特殊的地址,他是本地换回网络的IP,并且指定了服务器监听端口为8888.
然后调用ConServer发起向服务器的连接.如果连接成功,socket状态就会更新为ConnectedState.

注意只有socket的状态是连接状态(ConnectedState)的时候,我们才能向服务器发送数据和接收数据.

当我们发送数据后,调用DisconServer测试一下是否能正常断开服务器连接.在实际工作中,我们不必在发送后立刻断开网络连接.
如果发送成功,服务器会收到"8675"这个字符串.



## 从服务器接收数据

当socket的状态是连接状态(ConnectedState)的时候,我们可以随时接收来自服务器的数据.同样因为我们并不知道服务什么时候发送数据给客户端,所以我们需要检查和等待服务器.


在服务器客户端通讯中,有两种协调工作方式,同步模式和异步模式.

### 同步模式

就是发起发送或者接收后,以循环的方式不停检查处理结果,直到获得处理结果后才进行下一步的工作.我们上面的所有代码都是按照,同步模式来写的.
同步模式逻辑上比较简单,比较容易理解,所以在工作量不太大的情况下,都可以用同步模式开展工作.

### 异步模式

就是发起发送或者接收后,不管处理结果,立刻返回去做别的事情,当收到处理结果的信号后,再在槽函数中处理后续工作.
当理解同步模式后,异步模式就比较容易理解了,在需要大量的并行工作时,异步工作模式是必选.


### 如果进行同步模式数据接收


```
 while(1)
        if(tc.tcpClient->state() == QAbstractSocket::ConnectedState)
            if( tc.tcpClient->waitForReadyRead())
            {
                qDebug()<<"read data from server.....";
                QByteArray buffer = tc.tcpClient->readAll();//读取所有数据
                if(!buffer.isEmpty())
                {
                    qDebug()<<"data size "<<buffer.size();
                    qDebug()<<buffer;
                }
            }
            else
                qDebug()<<"read data timeout or error. try again";
        else
            break;
```

由于我们不知道服务器什么时候发送数据,所以我们需要将程序阻塞在waitForReadyRead函数里.当客户端发现服务器发送了数据或者超时(默认是30秒)后,该函数返回.同样当函数返回true说明接收成功,否则返回false.

需要注意的是QT的socket会自动将接收到的数据放在缓冲区中,所以我们实际上是从缓存区中读取来自服务器的数据.但是缓冲区的大小是有限的,所以我们要尽可能快的把数据从缓冲区取走.不能取走缓冲区数据可能导致数据溢出错误.

和发送数据相类似我们可以使用read函数来读取数据,该函数的定义如下:

QByteArray read(qint64 maxlen);

其中maxlen是指我们一次最多可以读取多少个字节的数据.最终读取到的数据可能小于这个数值.这意味着我们读取数据可能需要多次循环才能读完.
为了简化操作,QT提供了QByteArray readAll();函数.这个函数可以一次性读完目前收到的所有数据.





# 用QT实现TCP服务器

相比于客户端,服务器的工作略微复杂一些.通常服务器端的工作流程如下:

绑定端口->开始侦听->收到连接请求->分配通讯端口->在通讯端口上开始通讯


```
class TcpServer:public QObject
{
public:
    TcpServer(QWidget *parent = 0);

    ~TcpServer();
public:
    QHostAddress ip;
    quint16 port;
    bool SvStart();
    qint64 SendDataAll(const char * data, qint64 len);
    qint64 SendData(QTcpSocket *tcpClient,const char * data, qint64 len);
private:
    QTcpServer *tcpServer;
    QList<QTcpSocket*> tcpClient;
    QTcpSocket *currentClient;

public slots:
    void NewConnectionSlot();
    void onDisconnectedSlot();
    void onReadData();
};
```

通过上面服务器端的类的定义可以看到,除了在客户端已经使用过的ip地址和port(端口号)之外,还需要QList<QTcpSocket*> tcpClient;这个变量用来保存连接到当前服务器的各个socket的列表.
对于客户端来说,只需要保存自己当前的socket就可以了,但是对于服务器来说,可能有多个client连接到服务器,所以需要保存所有client的socket,以便对每个client分别通讯.



## 服务器初始化

```
    QList<QHostAddress> localaddrs;
    TcpServer(QWidget *parent = 0):QObject(parent)
    {
        tcpServer = new QTcpServer(this);
        //获得本机可用地址清单.
        localaddrs = QNetworkInterface::allAddresses();
        foreach(QHostAddress ha, localaddrs)
        {
            qDebug()<<ha.toString();
        }
        ip="127.0.0.1";//本地loop地址
        port=8888;

        connect(tcpServer, &QTcpServer::newConnection, this, &TcpServer::NewConnectionSlot);
    }
```

在我们开启服务器侦听之前首先需要初始一些服务必要的数据,首先需要建立一个QTcpServer的实例,用来实现服务器本身.同时我们需要设定服务器绑定的ip地址和端口.

通常情况下,我们的电脑有多个网口(比如有线网和无线网)分别对应不同的ip地址,所以需要指定我们当前的服务器绑定那个ip地址.我们默认指定的地址127.0.0.1是对内部的本地回环地址.如果是对外的地址,需要修改ip这个属性.
同时我们需要指定当前服务器绑定的侦听端口是什么.一个电脑是可以允许多个服务存在的,只要不同的服务器绑定不同的端口就行了.与此同时,服务器绑定的端口是侦听端口,也就是说这里是用来等待客户端连接的,当客户和服务器建立连接后,服务器会为每个客户端再分配一个通讯的端口,以后的通讯过程都是使用通讯端口来开展的,侦听端口还是用于后续其他客户端连接而等待连接请求.

除了上述参数,我们还需要设定一个信号和槽的连接,当服务器收到新的客户端请求建立连接后,newConnection会被触发.此时新的连接已经建立,但是为了方便日后服务器开展后续通讯,通常需要在这个信号的响应函数中保存当前的socket信息.


### 发起侦听

```
bool TcpServer::SvStart()
{
    qDebug()<<"try to start tcpserver at"<<ip.toString()<<":"<<port;
    bool ok = tcpServer->listen(ip, port);
     if(ok)
         qDebug("tcp server started!");
     else
         qDebug("tcp server start failed!");
     return ok;
}
```

侦听就是服务器一直监听某个端口,等待客户端发送连接请求,当双方建立连接后,就会分配到其他端口进行实际的数据通讯.


### 响应新连接

当客户端向服务器发起连接请求并成功连接后,QTcpServer就会emmit一个QTcpServer::newConnection信号,我们在之前的构造函数中已经为这个信号关联了槽函数:
>connect(tcpServer, &QTcpServer::newConnection, this, &TcpServer::NewConnectionSlot);

所以客户端连接成功后会调用如下函数

```
void TcpServer::NewConnectionSlot()
{
    qDebug()<<"New Connection Slot coming...";
    currentClient = tcpServer->nextPendingConnection();
    tcpClient.append(currentClient);
    connect(currentClient, &QIODevice::readyRead, this, &TcpServer::onReadData);
    connect(currentClient, &QAbstractSocket::disconnected, this, &TcpServer::onDisconnectedSlot);

    qDebug()<<"ip:"<<currentClient->peerAddress().toString()<<":"<<currentClient->peerPort();
}
```

因为服务器可能会连接多个客户端,所以我们将当前连接成功的socket保存到tcpClient这个列表中.同时为这个socekt关联数据到达信号readyRead和连接断开信号disconnected相应的槽函数.




### 接收数据

与连接成功类似,当服务器收到来自客户端的数据后,QTcpSocket会发出一个readyRead的信号.为什么不是从QTcpServer发出的呢?因为读写实际上是发生在套件字上的,TCPServer上一个管理调度的角色,负责端口侦听,分配端口,建立和断开连接等.所以读写所需的相关信号都是由Socket发出的.readyRead是由QTcpSocket的父类的父类定义的.相关定义如下:
>class Q_NETWORK_EXPORT QTcpSocket : public QAbstractSocket
>>class Q_NETWORK_EXPORT QAbstractSocket : public QIODevice
>>>QIODevice::readyRead

由于我们已经关联了对应的槽函数,所以当有可读数据的时候,就会进入下面的函数.

```
void TcpServer::onReadData()
{
    qDebug()<<"onReadData";
    // 遍历所有客户端,批量处理
    for(int i=0; i<tcpClient.length(); i++)
    {
        QByteArray buffer = tcpClient[i]->readAll();
        if(buffer.isEmpty())    continue;

        static QString IP_Port, IP_Port_Pre;
        qDebug()<<"read data ip:"<<currentClient->peerAddress().toString()<<":"<<currentClient->peerPort();
        qDebug()<<buffer;
    }

}
```


在上面这个函数里,我们是通过readAll来简化读取的过程,实际上也可以直接使用read函数来读取.
读取数据有两种方法,第一种是一旦进入读取函数就批量处理所有的socket,这样做的好处是当连接大量客户端并且频繁进行数据收发的时候能够节省进入槽的次数.上面的程序就是按第一种方法做的.
第二种方法是根据发送信号的sender,之间转换成socket进行通讯.

```
void TcpServer::onReadData()
{
    qDebug()<<"onReadData";

    //方式2:
    //根据发送信号的socket直接读取数据
    QTcpSocket *socket=qobject_cast<QTcpSocket *>(sender());
    qDebug()<<"from socket "<<socket->peerPort();
    QByteArray buffer = socket->readAll();
    static QString IP_Port, IP_Port_Pre;
    qDebug()<<"read data ip:"<<socket->peerAddress().toString()<<":"<<socket->peerPort();
    qDebug()<<buffer;

}
```



### 发送数据

这部分很简单,和客户端写法基本一致,直接通过套接字的write函数发送数据就可以了,返回值是实际发送数据的数量.

```
qint64 TcpServer::SendData(QTcpSocket *tcpClient,const char * data, qint64 len)
{
    qint64 slen;
    slen=tcpClient->write(data,len); //qt5除去了.toAscii()
    qDebug()<<"SendData len:"<<slen<<"data:"<<data;
    return slen;
}
```


### 断开连接

与接收数据类似,我们在新连接建立的时候为对应的套接字关联了断开信号对应的槽函数:
>connect(currentClient, &QAbstractSocket::disconnected, this, &TcpServer::onDisconnectedSlot);

下面的槽函数也是批量处理,将所有的套接字遍历了一遍,然后将状态已经是QAbstractSocket::UnconnectedState套接字从列表中删除.
```
void TcpServer::onDisconnectedSlot()
{
    qDebug()<<"onDisconnectedSlot";
    //由于disconnected信号并未提供SocketDescriptor，所以需要遍历寻找
    for(int i=0; i<tcpClient.length(); i++)
    {
        if(tcpClient[i]->state() == QAbstractSocket::UnconnectedState)
        {
            qDebug()<<"disconnect ip:"<<currentClient->peerAddress().toString()<<":"<<currentClient->peerPort();
            // 删除存储在tcpClient列表中的客户端信息
             tcpClient[i]->destroyed();
             tcpClient.removeAt(i);
        }
    }
}
```


# 作业

我们已经学习了如何建立客户端和服务器,结合之前学习的内容,请建立如下系统:
### 客户端
通过定时器不断发送数据给服务器,同时接收服务器发回来的数据.
### 服务器
能够接收多个客户端发送的数据,然后将客户端发送的数据后面追加上该客户端套接字对应的端口号,发回给对应的客户端.

例如:
>服务器绑定监听端口8888,当客户端连接服务器后自动分配到了53421端口,然后客户端定时向服务器发送数据"client",服务器收到数据后给该客户端发送数据"client53421"